In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[-0.0461,  0.0470, -0.0572,  ...,  0.0272, -0.0968, -0.1026],
        [ 0.0644, -0.0326,  0.0298,  ..., -0.0492,  0.0148, -0.0917],
        [-0.0349,  0.0044,  0.1276,  ...,  0.0631, -0.0094, -0.1470],
        ...,
        [-0.0611, -0.0486,  0.0876,  ..., -0.0890,  0.0159, -0.0135],
        [-0.0616,  0.0404, -0.0835,  ..., -0.0583,  0.2525, -0.0169],
        [-0.0239, -0.0226, -0.0159,  ..., -0.0669,  0.0884, -0.0524]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 150
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-66.5423, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-9.0924, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3045, device='cuda:0', grad_fn=<NegBackward0>), tensor(-49.1454, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-40.0023, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.3327, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.2003, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.4693, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-40.0023, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.3327, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.2003, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.4694, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.5967, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.0345, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.1834, device='cuda:0', grad_fn=<NegBackward0>), tensor(-51.3789, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.5968, device='cuda:0', grad_fn=<NegBackward0>) [tenso

tensor(-129.0195, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.1139, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.2740, device='cuda:0', grad_fn=<NegBackward0>), tensor(-97.6316, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-129.0195, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.1139, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.2740, device='cuda:0', grad_fn=<NegBackward0>), tensor(-97.6316, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-127.9477, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.0389, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.0987, device='cuda:0', grad_fn=<NegBackward0>), tensor(-96.8100, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-127.9477, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.0389, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.0987, device='cuda:0', grad_fn=<NegBackward0>), tensor(-96.8100, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-129.6973, device='cuda:0', grad_fn=<NegBackw

tensor(-153.4037, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.2300, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.9737, device='cuda:0', grad_fn=<NegBackward0>), tensor(-117.1999, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-152.7773, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.2494, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.9556, device='cuda:0', grad_fn=<NegBackward0>), tensor(-116.5722, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-152.7773, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.2494, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.9556, device='cuda:0', grad_fn=<NegBackward0>), tensor(-116.5722, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-151.0617, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.1239, device='cuda:0', grad_fn=<NegBackward0>), tensor(-14.7615, device='cuda:0', grad_fn=<NegBackward0>), tensor(-115.1763, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-151.0616, device='cuda:0', grad_fn=<NegB

tensor(-166.9787, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.0181, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.2183, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.7423, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-166.9787, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.0181, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.2183, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.7423, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-166.6713, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.0324, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.1717, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.4673, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-166.6713, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.0324, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.1717, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.4672, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-163.2166, device='cuda:0', grad_fn=<NegB

tensor(-172.3098, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.9203, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.7844, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.6050, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.8210, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.0405, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.9340, device='cuda:0', grad_fn=<NegBackward0>), tensor(-130.8465, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.8210, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.0405, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.9340, device='cuda:0', grad_fn=<NegBackward0>), tensor(-130.8465, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.4541, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.9797, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.7956, device='cuda:0', grad_fn=<NegBackward0>), tensor(-130.6789, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.4541, device='cuda:0', grad_fn=<NegB

tensor(-176.8577, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8569, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.4880, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.5127, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-176.8577, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8569, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.4880, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.5127, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-176.4978, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8937, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.4935, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.1106, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-176.4978, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8937, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.4935, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.1106, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-174.6733, device='cuda:0', grad_fn=<NegB

tensor(-174.9577, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.2159, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.5237, device='cuda:0', grad_fn=<NegBackward0>), tensor(-132.2181, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-177.7137, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.5326, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.9619, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.2193, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-177.7137, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.5326, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.9619, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.2192, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-179.7068, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.6466, device='cuda:0', grad_fn=<NegBackward0>), tensor(-18.1382, device='cuda:0', grad_fn=<NegBackward0>), tensor(-135.9221, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-179.7068, device='cuda:0', grad_fn=<NegB

tensor(-163.8981, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.2676, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.3567, device='cuda:0', grad_fn=<NegBackward0>), tensor(-123.2739, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-163.8981, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.2676, device='cuda:0', grad_fn=<NegBackward0>), tensor(-16.3567, device='cuda:0', grad_fn=<NegBackward0>), tensor(-123.2738, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-169.8441, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0069, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.0686, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.7686, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-169.8441, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0069, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.0686, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.7686, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-173.3051, device='cuda:0', grad_fn=<NegB

tensor(-184.4409, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-27.0020, device='cuda:0', grad_fn=<NegBackward0>), tensor(-19.0889, device='cuda:0', grad_fn=<NegBackward0>), tensor(-138.3500, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-181.8851, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.7047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-18.8734, device='cuda:0', grad_fn=<NegBackward0>), tensor(-136.3070, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-181.8851, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.7047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-18.8734, device='cuda:0', grad_fn=<NegBackward0>), tensor(-136.3070, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-180.6574, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.6646, device='cuda:0', grad_fn=<NegBackward0>), tensor(-18.4802, device='cuda:0', grad_fn=<NegBackward0>), tensor(-135.5126, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-180.6574, device='cuda:0', grad_fn=<NegB

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.09s/it]


----------------------------
Train_z, Test_z
PCA: 0.8058
SVD: 0.8065
NetD, Acc at first 0.8624
